In [1]:
%pip install tensorflow matplotlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import os

In [3]:
# Set up data paths
train_dir = 'data/train/'
test_dir = 'data/test/'

In [4]:
# Image data generator for real-time data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,  # Increase rotation range
    width_shift_range=0.3,  # Increase shift range
    height_shift_range=0.3,
    shear_range=0.3,  # Increase shear range
    zoom_range=0.3,  # Increase zoom range
    horizontal_flip=True,
    vertical_flip=True,  # Add vertical flip
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)


In [5]:
# Create generators for training and test datasets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Resize images to a common size
    batch_size=32,
    class_mode='categorical'  # Since it's a multiclass classification
)

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

Found 1211 images belonging to 4 classes.
Found 99 images belonging to 4 classes.


In [6]:
# Generate the model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.6),  # Dropout for regularization
    layers.Dense(len(train_generator.class_indices), activation='softmax')  # Output layer for multiclass
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


C:\Users\hirun\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
%pip install scipy


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [9]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 22s 599ms/step - accuracy: 0.5546 - loss: 1.1109 - val_accuracy: 0.5104 - val_loss: 1.1694
Epoch 2/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.6250 - loss: 1.0935 - val_accuracy: 0.5521 - val_loss: 1.1535
Epoch 3/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 19s 521ms/step - accuracy: 0.5971 - loss: 1.0334 - val_accuracy: 0.5208 - val_loss: 1.1596
Epoch 4/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.5625 - loss: 1.0789 - val_accuracy: 0.5208 - val_loss: 1.1451
Epoch 5/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 19s 503ms/step - accuracy: 0.5900 - loss: 0.9819 - val_accuracy: 0.6042 - val_loss: 1.1901
Epoch 6/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.6250 - loss: 1.0472 - val_accuracy: 0.5729 - val_loss: 1.2276
Epoch 7/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 19s 503ms/step - accuracy: 0.5738 - loss: 1.0546 - val_accuracy: 0.5625 - val_loss: 1.1427
Epoch 8/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.5625 - loss: 0.9959 - val_accuracy: 0.

In [10]:
test_loss, test_acc = model.evaluate(validation_generator)
print(f"Test accuracy: {test_acc}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.6681 - loss: 0.9604 
Test accuracy: 0.6363636255264282


In [12]:
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_snake(image_path):
    img = image.load_img(image_path, target_size=(150, 150))
    img_array = image.img_to_array(img)  # Convert image to array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize the image

    predictions = model.predict(img_array)
    class_idx = np.argmax(predictions[0])
    class_label = list(train_generator.class_indices.keys())[class_idx]
    return class_label

# Example usage
result = predict_snake(r'data/test/Ahaetulla nasuta/6f43be85218f44e681957b7b7c9bd2c2.jpg')
print(f'The snake species is: {result}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
The snake species is: Ahaetulla nasuta


In [13]:
import json

In [14]:
# Save the model
os.makedirs("models", exist_ok=True)
model.save("models/snakeimageclassifier.keras")

In [15]:
# Save class names as a JSON array
snake_names = list(train_generator.class_indices.keys())  # Extract class names as an array
with open("models/snake_class_names.json", "w") as f:
    json.dump(snake_names, f)

print("Model and class names saved successfully!")

Model and class names saved successfully!
